# OvertureMaestro Basic Usage

**OvertureMaestro** exposes some basic functions in the main Python module. Full documentation for them is available [here](../../api/OvertureMaestro/).

This notebook will show how to use the library in a few simple scenarios.

To learn more about `CLI` usage, see [this page](../command_line_interface/). The `help` page for the CLI is available [here](../../api/CLI/).

In [ ]:
import overturemaestro as om

## Get the data for a region to GeoDataFrame

### Without release version

If release version is omitted - **OvertureMaestro** automatically searches for the newest available version and applies it.

In [ ]:
london_bbox = (-0.120077, 51.498164, -0.090809, 51.508849)

In [ ]:
buildings_gdf = om.convert_bounding_box_to_geodataframe(
    theme="buildings", type="building", bbox=london_bbox
)
buildings_gdf

### With release version

Release version can also be passed explicitly.

In [ ]:
overture_maps_release = "2024-08-20.0"

In [ ]:
roads_gdf = om.convert_bounding_box_to_geodataframe(
    release=overture_maps_release, theme="transportation", type="segment", bbox=london_bbox
)
roads_gdf

### With PyArrow filtering
**OvertureMaestro** enables passing optional PyArrow filter used during data acquisition.

In [ ]:
water_gdf = om.convert_bounding_box_to_geodataframe(
    theme="base", type="water", bbox=london_bbox, pyarrow_filter=[("subtype", "=", "river")]
)
water_gdf

Colours inspired by https://snazzymaps.com/style/144886

In [ ]:
ax = buildings_gdf.plot(color="#f7cda9", zorder=3)
roads_gdf.plot(ax=ax, color="#cc7578", zorder=1)
water_gdf.plot(ax=ax, color="#97b5bf", zorder=2)
ax.set_xlim([london_bbox[0], london_bbox[2]])
ax.set_ylim([london_bbox[1], london_bbox[3]])
ax.set_axis_off()

### Result file sorting
By default, **OvertureMaestro** sorts the result file by geometry using Hilbert curve to make it smaller. It adds some time to the overall execution, but can reduce the file size. The files are already heavily compressed using ZSTD algorithm with max compression level (22) and sorting can improve spatial operations speed. 

Sorting can be disabled by the user.

In [ ]:
unsorted_pq = om.convert_geometry_to_parquet(
    theme="buildings",
    type="building",
    geometry_filter=om.geocode_to_geometry("City of London"),
    sort_result=False,
    ignore_cache=True,
)

In [ ]:
sorted_pq = om.convert_geometry_to_parquet(
    theme="buildings",
    type="building",
    geometry_filter=om.geocode_to_geometry("City of London"),
    sort_result=True,
    ignore_cache=True,
)

In [ ]:
import geopandas as gpd
from matplotlib import pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

gpd.read_parquet(unsorted_pq).reset_index().reset_index().plot(column="index", ax=ax1, cmap="jet")
gpd.read_parquet(sorted_pq).reset_index().reset_index().plot(column="index", ax=ax2, cmap="jet")

unsorted_size = unsorted_pq.stat().st_size
sorted_size = sorted_pq.stat().st_size

ax1.set_title(f"Unsorted: {unsorted_size} bytes")
ax2.set_title(
    f"Sorted: {sorted_size} bytes ({100 - (100 * sorted_size) / unsorted_size:.2f}% reduction)"
)

plt.show()

## Metadata functions

OvertureMaestro exposes some metadata functions for the user.

### Get all available release versions

List of available releases is downloaded from the accompanying [GitHub repository](https://github.com/kraina-ai/overturemaps-releases-indexes/) with precalculated indexes.

In [ ]:
om.get_available_release_versions()

### Get newest available releave version

In [ ]:
om.get_newest_release_version()

### List all available theme/type pairs

Overture Maps data uses `themes` and `types` to partition the dataset by function. User can retrieve this list for any available release version supported by **OvertureMaestro**.

Full definition is explained in the [official schema documentation](https://docs.overturemaps.org/schema/concepts/).

If release index isn't cached locally, it will be loaded from the accompanying GitHub repository.

In [ ]:
om.get_available_theme_type_pairs(overture_maps_release)

Release version can also be infered implicitly - it will automatically load newest release version.

In [ ]:
om.get_available_theme_type_pairs()